<a href="https://colab.research.google.com/github/pujitha014/Diabetics_prediction_ensemblelearning/blob/main/random_diabetes2_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

df=pd.read_csv("/content/diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [8]:
x=df.drop("Outcome",axis=1).values
y=df["Outcome"].values

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x = scaler.fit_transform(x)
print(x)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [10]:
from sklearn.svm import SVC
import numpy as np

svm_classifier = SVC(kernel='rbf',gamma='auto')
svm_classifier.fit(x, y)

svm_predictions = svm_classifier.predict(x)

svm_accuracy = np.mean(svm_predictions==y)
print(f"SVM Accuracy: {svm_accuracy}")

SVM Accuracy: 0.82421875


In [11]:
print(x)
x= np.column_stack((x, svm_predictions))


[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [13]:
import numpy as np
import matplotlib.pyplot as plt

class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate

        # Initialize weights and biases
        self.w1 = np.random.randn(input_nodes, hidden_nodes)
        self.b1 = np.ones((1, hidden_nodes))
        self.w2 = np.random.randn(hidden_nodes, output_nodes)
        self.b2 = np.ones((1, output_nodes))


    def relu(self, x):
        return 1 / (1 + np.exp(-x))

    def derivative(self, x):
        return x * (1 - x)

    def bound(self, x):
        return np.clip(x, -1, 1)

    def train(self, x_train, y_train, epochs=1000):
        lt = np.array([])
        for j in range(epochs):
            hidden_o = self.relu(np.dot(x_train, self.w1) + self.b1)
            final_o = self.relu(np.dot(hidden_o, self.w2) + self.b2)

            output_error = final_o - y_train
            error = np.mean((y_train - final_o) ** 2)

            final_e = self.derivative(final_o) * (output_error)
            hidden_e = np.dot(final_e, self.w2.T) * self.derivative(hidden_o)

            gradient_final_o = np.dot(hidden_o.T, final_e)
            gradient_hidden_o = np.dot(x_train.T, hidden_e)

            max_gradient_norm = 1.0
            if np.linalg.norm(gradient_final_o) > max_gradient_norm:
                gradient_final_o = gradient_final_o * max_gradient_norm / np.linalg.norm(gradient_final_o)
            if np.linalg.norm(gradient_hidden_o) > max_gradient_norm:
                gradient_hidden_o = gradient_hidden_o * max_gradient_norm / np.linalg.norm(gradient_hidden_o)

            self.w2 = self.bound(self.w2 - self.learning_rate * np.where(gradient_final_o < 0, gradient_final_o, 0))
            self.w1 = self.bound(self.w1 - self.learning_rate * np.where(gradient_hidden_o < 0, gradient_hidden_o, 0))

            self.b2 = self.bound(self.b2 - self.learning_rate * final_e.sum(axis=0, keepdims=True))
            self.b1 = self.bound(self.b1 - self.learning_rate * hidden_e.sum(axis=0, keepdims=True))

            lt = np.append(lt, error)

            # if j % 100 == 0:
            #     print(f"Epoch {j}, Loss: {error}")
        return lt

    def predict(self, x_test):
        hidden_o = self.relu(np.dot(x_test, self.w1) + self.b1)
        final_o = np.dot(hidden_o, self.w2) + self.b2
        predictions = np.where(final_o >= 0.5, 1, 0)
        return predictions


# input_nodes = x_train.shape[1]
# hidden_nodes = 12
# output_nodes = 1
# learning_rate = 0.01

# accur2=[]
# for i in range(10):
#       print(i)
#       nn = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
#       losses = nn.train(x_train, y_train.reshape(-1, 1))
#       predictions = nn.predict(x_test)
#       accuracy = np.mean(predictions == y_test.reshape(-1, 1))
#       # print(f"ANN Accuracy: {accuracy}")
#       accur2.append(accuracy)

# print(accur2)
# print(np.mean(accur2))

# if run just ann use 2000 iterations else 1000

In [21]:
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def bagged():
    n = 10

    models = []
    sample_size = len(x_train)
    print(sample_size)

    for i in range(n):
        indices = np.random.choice(sample_size, sample_size, replace=True)
        x_bootstrap = x_train[indices]
        y_bootstrap = y_train[indices]

        model = NeuralNetwork(x_bootstrap.shape[1],12,1,0.01)
        model.train(x_bootstrap,y_bootstrap.reshape(-1, 1))

        models.append(model)

    predictions = []
    accuracies=[]
    for model in models:
        y_pred = model.predict(x_test)
        predictions.append(y_pred)
        acc=np.mean(y_test.reshape(-1,1)==y_pred)
        accuracies.append(acc)
    return predictions,accuracies

t1=[]
t2=[]
for i in range(3):
    predictions,accuracies=bagged()
    ensemble_predictions = np.round(np.mean(predictions, axis=0)) >= 0.5
    # temp=y_test.reshape(-1,1)
    ensemble_accuracy = np.mean(y_test.reshape(-1,1)==ensemble_predictions)

    t1.append(ensemble_accuracy)
    t2.append(max(accuracies))
    print(accuracies)

print(f"Ensemble Accuracy: {np.mean(t1)}")
print(f"max: {np.mean(t2)}")

614
[0.7532467532467533, 0.8051948051948052, 0.7727272727272727, 0.7727272727272727, 0.7792207792207793, 0.7987012987012987, 0.7922077922077922, 0.7727272727272727, 0.7337662337662337, 0.7727272727272727]
614
[0.7727272727272727, 0.7987012987012987, 0.7792207792207793, 0.7857142857142857, 0.7922077922077922, 0.7987012987012987, 0.8116883116883117, 0.7857142857142857, 0.7532467532467533, 0.7857142857142857]
614
[0.7987012987012987, 0.7987012987012987, 0.7857142857142857, 0.7987012987012987, 0.7857142857142857, 0.7727272727272727, 0.7467532467532467, 0.7402597402597403, 0.7662337662337663, 0.7857142857142857]
Ensemble Accuracy: 0.8030303030303031
max: 0.8051948051948052
